In [ ]:
table_descriptions = {

    "Departments": "The Departments table holds academic department information. Each department may be associated with multiple students, professors, and courses. Use it when filtering by department or analyzing department-specific activity.",



    "Students": "The Students table contains personal and academic enrollment details for students. Each student is associated with a department and can be enrolled in multiple courses. Use this table to filter students by department or analyze student enrollment.",



    "Professors": "The Professors table stores data about academic faculty. Each professor belongs to a department and may teach one or more courses. Use this when identifying course instructors or analyzing departmental faculty.",



    "Courses": "The Courses table contains the list of academic courses offered by departments. Each course is taught by a professor and belongs to a department. Use it for curriculum planning, instructor assignments, or course scheduling.",



    "Enrollments": "The Enrollments table tracks which students are enrolled in which courses, including the date of enrollment. Use it to analyze student course load, popularity of courses, or to join student and course data.",



    "Classrooms": "The Classrooms table defines physical classrooms, including building and room number, and capacity. It supports the scheduling system for courses. Use it to analyze classroom utilization or room assignments.",



    "Schedules": "The Schedules table links courses to classrooms and timeslots, including day of the week and start/end times. Use it to retrieve timetable information or detect scheduling conflicts.",



    "Grades": "The Grades table records student performance in courses by linking enrollment to a grade and grade point. Use it for GPA calculation, performance tracking, and academic reports.",



    "Assignments": "The Assignments table contains homework or project records tied to specific courses. Each assignment has a title, due date, and total marks. Use this table for academic workload or submission tracking.",



    "Submissions": "The Submissions table tracks which students submitted which assignments and the marks they received. Use this to evaluate assignment performance and submission timing.",



    "Clubs": "The Clubs table holds information about student clubs, including their faculty advisor and founding year. Use this when analyzing extracurricular involvement or managing student organizations.",



    "ClubMembers": "The ClubMembers table links students to clubs and records their role (e.g., President, Member). Use this table to retrieve club rosters, roles, or member count."

}   

table_metadata = {

    "Departments": {

        "name": "Departments",

        "type": "table",

        "joins_with": ["Students", "Professors", "Courses"],

        "primary_keys": ["department_id"],

        "foreign_keys": [],

        "topics": ["academic departments", "head of department"]

    },

    "Students": {

        "name": "Students",

        "type": "table",

        "joins_with": ["Departments", "Enrollments", "Submissions", "ClubMembers"],

        "primary_keys": ["student_id"],

        "foreign_keys": ["department_id → Departments(department_id)"],

        "topics": ["student info", "enrollment", "academic records"]

    },

    "Professors": {

        "name": "Professors",

        "type": "table",

        "joins_with": ["Departments", "Courses", "Clubs"],

        "primary_keys": ["professor_id"],

        "foreign_keys": ["department_id → Departments(department_id)"],

        "topics": ["faculty", "academic staff", "course instructor"]

    },

    "Courses": {

        "name": "Courses",

        "type": "table",

        "joins_with": ["Departments", "Professors", "Enrollments", "Schedules", "Assignments"],

        "primary_keys": ["course_id"],

        "foreign_keys": [

            "department_id → Departments(department_id)",

            "professor_id → Professors(professor_id)"

        ],

        "topics": ["course catalog", "teaching", "curriculum"]

    },  "Enrollments": {

        "name": "Enrollments",

        "type": "table",

        "joins_with": ["Students", "Courses", "Grades"],

        "primary_keys": ["enrollment_id"],

        "foreign_keys": [

            "student_id → Students(student_id)",

            "course_id → Courses(course_id)"

        ],

        "topics": ["course registration", "enrollment", "student load"]

    },

    "Classrooms": {

        "name": "Classrooms",

        "type": "table",

        "joins_with": ["Schedules"],

        "primary_keys": ["classroom_id"],

        "foreign_keys": [],

        "topics": ["physical space", "room capacity", "locations"]

    },

    "Schedules": {

        "name": "Schedules",

        "type": "table",

        "joins_with": ["Courses", "Classrooms"],

        "primary_keys": ["schedule_id"],

        "foreign_keys": [

            "course_id → Courses(course_id)",

            "classroom_id → Classrooms(classroom_id)"

        ],

        "topics": ["timetables", "class schedule", "room booking"]

    },

    "Grades": {

        "name": "Grades",

        "type": "table",

        "joins_with": ["Enrollments"],

        "primary_keys": ["grade_id"],

        "foreign_keys": ["enrollment_id → Enrollments(enrollment_id)"],

        "topics": ["marks", "GPA", "student performance"]

    },

    "Assignments": {

        "name": "Assignments",

        "type": "table",

        "joins_with": ["Courses", "Submissions"],

        "primary_keys": ["assignment_id"],

        "foreign_keys": ["course_id → Courses(course_id)"],

        "topics": ["homework", "projects", "due dates"]

    },

    "Submissions": {

        "name": "Submissions",

        "type": "table",

        "joins_with": ["Assignments", "Students"],

        "primary_keys": ["submission_id"],

        "foreign_keys": [

            "assignment_id → Assignments(assignment_id)",

            "student_id → Students(student_id)"

        ],

        "topics": ["student work", "marks obtained", "submission date"]

    },

    "Clubs": {

        "name": "Clubs",

        "type": "table",

        "joins_with": ["Professors", "ClubMembers"],

        "primary_keys": ["club_id"],

        "foreign_keys": ["faculty_advisor_id → Professors(professor_id)"],

        "topics": ["student organizations", "extracurriculars"]

    },

    "ClubMembers": {

        "name": "ClubMembers",

        "type": "table",

        "joins_with": ["Clubs", "Students"],

        "primary_keys": ["member_id"],

        "foreign_keys": [

            "club_id → Clubs(club_id)",

            "student_id → Students(student_id)"

        ],

        "topics": ["student roles", "club participation"]

    }} 

In [1]:
import sqlite3

# Connect to the SQLite database. This creates the file if it doesn't exist.
conn = sqlite3.connect('university.db')
cursor = conn.cursor()

# ----------------------------------------------------
# 1. Create Tables
# ----------------------------------------------------

# Departments Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Departments (
        department_id INTEGER PRIMARY KEY,
        department_name TEXT NOT NULL,
        head_of_department_id INTEGER,
        founding_year INTEGER
    );
''')

# Professors Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Professors (
        professor_id INTEGER PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        department_id INTEGER,
        hire_date DATE NOT NULL,
        salary REAL NOT NULL,
        FOREIGN KEY (department_id) REFERENCES Departments(department_id)
    );
''')

# Students Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Students (
        student_id INTEGER PRIMARY KEY,
        first_name TEXT NOT NULL,
        last_name TEXT NOT NULL,
        email TEXT NOT NULL UNIQUE,
        department_id INTEGER,
        enrollment_date DATE NOT NULL,
        FOREIGN KEY (department_id) REFERENCES Departments(department_id)
    );
''')

# Courses Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Courses (
        course_id INTEGER PRIMARY KEY,
        course_name TEXT NOT NULL,
        course_code TEXT NOT NULL,
        department_id INTEGER,
        professor_id INTEGER,
        credits INTEGER NOT NULL,
        FOREIGN KEY (department_id) REFERENCES Departments(department_id),
        FOREIGN KEY (professor_id) REFERENCES Professors(professor_id)
    );
''')

# Enrollments Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Enrollments (
        enrollment_id INTEGER PRIMARY KEY,
        student_id INTEGER,
        course_id INTEGER,
        enrollment_date DATE NOT NULL,
        FOREIGN KEY (student_id) REFERENCES Students(student_id),
        FOREIGN KEY (course_id) REFERENCES Courses(course_id)
    );
''')

# Grades Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Grades (
        grade_id INTEGER PRIMARY KEY,
        enrollment_id INTEGER,
        grade_letter TEXT,
        grade_point REAL,
        grade_date DATE,
        FOREIGN KEY (enrollment_id) REFERENCES Enrollments(enrollment_id)
    );
''')

# Classrooms Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Classrooms (
        classroom_id INTEGER PRIMARY KEY,
        building TEXT NOT NULL,
        room_number TEXT NOT NULL,
        capacity INTEGER NOT NULL
    );
''')

# Schedules Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Schedules (
        schedule_id INTEGER PRIMARY KEY,
        course_id INTEGER,
        classroom_id INTEGER,
        day_of_week TEXT NOT NULL,
        start_time TIME NOT NULL,
        end_time TIME NOT NULL,
        FOREIGN KEY (course_id) REFERENCES Courses(course_id),
        FOREIGN KEY (classroom_id) REFERENCES Classrooms(classroom_id)
    );
''')

# Assignments Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Assignments (
        assignment_id INTEGER PRIMARY KEY,
        course_id INTEGER,
        assignment_title TEXT NOT NULL,
        due_date DATE NOT NULL,
        total_marks INTEGER NOT NULL,
        FOREIGN KEY (course_id) REFERENCES Courses(course_id)
    );
''')

# Submissions Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Submissions (
        submission_id INTEGER PRIMARY KEY,
        assignment_id INTEGER,
        student_id INTEGER,
        submission_date DATE NOT NULL,
        marks_obtained INTEGER,
        FOREIGN KEY (assignment_id) REFERENCES Assignments(assignment_id),
        FOREIGN KEY (student_id) REFERENCES Students(student_id)
    );
''')

# Clubs Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Clubs (
        club_id INTEGER PRIMARY KEY,
        club_name TEXT NOT NULL,
        faculty_advisor_id INTEGER,
        founding_year INTEGER,
        FOREIGN KEY (faculty_advisor_id) REFERENCES Professors(professor_id)
    );
''')

# ClubMembers Table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS ClubMembers (
        member_id INTEGER PRIMARY KEY,
        student_id INTEGER,
        club_id INTEGER,
        role TEXT,
        join_date DATE NOT NULL,
        FOREIGN KEY (student_id) REFERENCES Students(student_id),
        FOREIGN KEY (club_id) REFERENCES Clubs(club_id)
    );
''')

# ----------------------------------------------------
# 2. Add Data
# ----------------------------------------------------

# Sample data
departments_data = [
    (1, 'Computer Science', 101, 1985),
    (2, 'Mechanical Engineering', 102, 1960),
    (3, 'Business Administration', 103, 1975)
]

professors_data = [
    (101, 'Emily', 'Carter', 'emily.carter@univ.edu', 1, '2010-08-15', 120000.00),
    (102, 'David', 'Chen', 'david.chen@univ.edu', 2, '2005-01-20', 150000.00),
    (103, 'Sarah', 'Miller', 'sarah.miller@univ.edu', 3, '2012-05-10', 110000.00),
    (104, 'John', 'Smith', 'john.smith@univ.edu', 1, '2018-09-01', 95000.00)
]

students_data = [
    (1001, 'Alice', 'Johnson', 'alice.johnson@univ.edu', 1, '2023-09-01'),
    (1002, 'Bob', 'Williams', 'bob.williams@univ.edu', 2, '2022-09-01'),
    (1003, 'Charlie', 'Brown', 'charlie.brown@univ.edu', 1, '2023-09-01'),
    (1004, 'Diana', 'Prince', 'diana.prince@univ.edu', 3, '2021-09-01')
]

courses_data = [
    (201, 'Introduction to Programming', 'CS101', 1, 101, 4),
    (202, 'Thermodynamics', 'ME205', 2, 102, 3),
    (203, 'Principles of Marketing', 'BA301', 3, 103, 3),
    (204, 'Data Structures', 'CS202', 1, 104, 4),
    (205, 'Software Engineering', 'CS305', 1, 101, 4)
]

enrollments_data = [
    (3001, 1001, 201, '2024-01-15'),
    (3002, 1002, 202, '2024-01-15'),
    (3003, 1003, 201, '2024-01-15'),
    (3004, 1001, 204, '2024-01-16'),
    (3005, 1003, 204, '2024-01-16')
]

grades_data = [
    (4001, 3001, 'A', 4.00, '2024-05-20'),
    (4002, 3002, 'B+', 3.30, '2024-05-20'),
    (4003, 3003, 'B', 3.00, '2024-05-20')
]

classrooms_data = [
    (501, 'Science Hall', '101A', 50),
    (502, 'Engineering Building', '202B', 30),
    (503, 'Business School', '305C', 100)
]

schedules_data = [
    (6001, 201, 501, 'Monday', '09:00:00', '10:30:00'),
    (6002, 201, 501, 'Wednesday', '09:00:00', '10:30:00'),
    (6003, 202, 502, 'Tuesday', '11:00:00', '12:30:00'),
    (6004, 204, 501, 'Monday', '14:00:00', '15:30:00'),
    (6005, 204, 501, 'Wednesday', '14:00:00', '15:30:00')
]

assignments_data = [
    (7001, 201, 'First Programming Project', '2024-03-15', 100),
    (7002, 201, 'Midterm Exam', '2024-04-20', 100),
    (7003, 204, 'Data Structures Project', '2024-05-01', 100)
]

submissions_data = [
    (8001, 7001, 1001, '2024-03-14', 95),
    (8002, 7001, 1003, '2024-03-15', 88),
    (8003, 7002, 1001, '2024-04-20', 92)
]

clubs_data = [
    (901, 'Computer Science Club', 101, 2015),
    (902, 'Robotics Club', 102, 2018),
    (903, 'Business Society', 103, 2010)
]

club_members_data = [
    (10001, 1001, 901, 'President', '2024-02-01'),
    (10002, 1003, 901, 'Member', '2024-02-05'),
    (10003, 1002, 902, 'Member', '2024-03-10')
]

# Insert data into tables
def insert_data(table_name, data):
    # This dynamically builds the INSERT statement for each table
    placeholders = ', '.join(['?'] * len(data[0]))
    insert_stmt = f"INSERT INTO {table_name} VALUES ({placeholders})"
    cursor.executemany(insert_stmt, data)
    print(f"Inserted {len(data)} rows into {table_name}")

insert_data('Departments', departments_data)
insert_data('Professors', professors_data)
insert_data('Students', students_data)
insert_data('Courses', courses_data)
insert_data('Enrollments', enrollments_data)
insert_data('Grades', grades_data)
insert_data('Classrooms', classrooms_data)
insert_data('Schedules', schedules_data)
insert_data('Assignments', assignments_data)
insert_data('Submissions', submissions_data)
insert_data('Clubs', clubs_data)
insert_data('ClubMembers', club_members_data)

# Commit changes and close the connection
conn.commit()
conn.close()

print("Database 'university.db' created and populated successfully.")

Inserted 3 rows into Departments
Inserted 4 rows into Professors
Inserted 4 rows into Students
Inserted 5 rows into Courses
Inserted 5 rows into Enrollments
Inserted 3 rows into Grades
Inserted 3 rows into Classrooms
Inserted 5 rows into Schedules
Inserted 3 rows into Assignments
Inserted 3 rows into Submissions
Inserted 3 rows into Clubs
Inserted 3 rows into ClubMembers
Database 'university.db' created and populated successfully.
